In [1]:
import math 
import numpy as np 
import pybullet as p 
import random 
from numpy.random import randint 
import time 

In [2]:
cid = p.connect(p.SHARED_MEMORY)
if (cid<0):
	p.connect(p.GUI)
p.setPhysicsEngineParameter(enableFileCaching=0)
#reseting simulation paramaters
p.resetSimulation()
p.setGravity(0,0,-9.8)
useRealTimeSim = 1
p.setRealTimeSimulation(useRealTimeSim)

In [3]:


# there might be an error in the assignment of what is x1,x2,y1,y2 for collision test, beware.
tunnels = {"Straight":
           {"Orientation_Characteristics":
            {'Path_one': #xy-values from zero
            {"0":np.array([0,-2.5]),
             "1":np.array([2.5,0]),
             "-1":np.array([-2.5,0]),
             "3":np.array([0,2.5])
            }},
            "Block_relative_coordinates":
            #x1,x2 y1,y2 from zero
            {"0": np.array([[1.25,-1.25],[2.5,-2.5]]),
             "1":np.array([[2.5,-2.5],[1.25,-1.25]]),
             "-1": np.array([[2.5,-2.5],[1.25,-1.25]]),
             "3":np.array([[1.25,-1.25],[2.5,-2.5]])
           }},
           "CrossWay":
           {"Orientation_Characteristics":
            {'Path_one':
            {"0":np.array([0,-6.25]),
             "1":np.array([6.25,0]),
             "-1":np.array([-6.25,0]),
             "3":np.array([0,6.25])
            },
             'Path_two':
            {"0":np.array([-3.75,-2.5]),
             "1":np.array([2.5,-3.75]),
             "-1":np.array([-2.5,3.75]),
             "3":np.array([3.75,2.5])
            },
             'Path_three':
            {"0":np.array([3.75,-2.5]),
             "1":np.array([2.5,3.75]),
             "-1":np.array([-2.5,-3.75]),
             "3":np.array([-3.75,2.5])
            }},
            "Block_relative_coordinates":
            {"0": np.array([[[1.25,-1.25],[1.25,-6.25]],[[3.75,-3.75],[-1.25,-3.75]]]),
             "1":np.array([[[6.25,-1.25],[1.25,-1.25]],[[3.75,1.25],[3.75,-3.75]]]),
             "-1": np.array([[[1.25,-6.25],[1.25,-1.25]],[[-1.25,-3.75],[3.75,-3.75]]]),
             "3":np.array([[[1.25,-1.25],[6.25,-1.25]],[[3.75,-3.75],[3.75,1.25]]])
           }
           },
           "Downleft":
           {"Orientation_Characteristics":
            {'Path_one':
            {"0":np.array([0,-6.25]),
             "1":np.array([6.25,0]),
             "-1":np.array([-6.25,0]),
             "3":np.array([0,6.25])
            },
             'Path_two':
            {"0":np.array([-3.75,-2.5]),
             "1":np.array([2.5,-3.75]),
             "-1":np.array([-2.5,3.75]),
             "3":np.array([3.75,2.5])
            }},
            "Block_relative_coordinates":
            {"0": np.array([[[1.25,-1.25],[1.25,-6.25]],[[1.25,-3.75],[-1.25,-3.75]]]),
             "1":np.array([[[6.25,-1.25],[1.25,-1.25]],[[3.75,1.25],[1.25,-3.75]]]),
             "-1": np.array([[[1.25,-6.25],[1.25,-1.25]],[[-1.25,-3.75],[3.75,-1.25]]]),
             "3":np.array([[[1.25,-1.25],[6.25,-1.25]],[[3.75,-1.25],[3.75,1.25]]])
           }
           },
           "DownRight":
           {"Orientation_Characteristics":
            {'Path_one':
            {"0":np.array([0,-6.25]),
             "1":np.array([6.25,0]),
             "-1":np.array([-6.25,0]),
             "3":np.array([0,6.25])
            },
             'Path_three':
            {"0":np.array([3.75,-2.5]),
             "1":np.array([2.5,3.75]),
             "-1":np.array([-2.5,-3.75]),
             "3":np.array([-3.75,2.5])
            }},
            "Block_relative_coordinates":
            {
             "0": np.array([[[1.25,-1.25],[1.25,-6.25]],[[3.75,-1.25],[-1.25,-3.75]]]),
             "1":np.array([[[6.25,-1.25],[1.25,-1.25]],[[3.75,1.25],[3.75,-1.25]]]),
             "-1": np.array([[[1.25,-6.25],[1.25,-1.25]],[[-2.5,-5.0],[1.25,-3.75]]]),
             "3":np.array([[[1.25,-1.25],[6.25,-1.25]],[[1.25,-3.75],[3.75,1.25]]]) 
            }
           },
           "LeftRight":
           {"Orientation_Characteristics":
            {'Path_two':
            {"0":np.array([-3.75,-2.5]),
             "1":np.array([2.5,-3.75]),
             "-1":np.array([-2.5,3.75]),
             "3":np.array([3.75,2.5])
            },
             'Path_three':
            {"0":np.array([3.75,-2.5]),
             "1":np.array([2.5,3.75]),
             "-1":np.array([-2.5,-3.75]),
             "3":np.array([-3.75,2.5])
            }},
            "Block_relative_coordinates":
            {"0": np.array([[[1.25,-1.25],[1.25,-3.75]],[[3.75,-3.75],[-1.25,-3.75]]]),
             "1":np.array([[[3.75,-1.25],[1.25,-1.25]],[[3.75,1.25],[3.75,-3.75]]]),
             "-1": np.array([[[1.25,-3.75],[1.25,-1.25]],[[-1.25,-3.75],[3.75,-3.75]]]),
             "3":np.array([[[1.25,-1.25],[3.75,-1.25]],[[3.75,-3.75],[3.75,1.25]]])
           }
           }
           }

In [4]:
def correct_position(g_o,key,xyz2):
    if key == 'Straight':
        if g_o == '0':
            xyz2[1]=xyz2[1]-2.5
        elif g_o == '1':
            xyz2[0]=xyz2[0]+2.5
        elif g_o == '-1':
            xyz2[0]=xyz2[0]-2.5
        elif g_o == '3':
            xyz2[1]=xyz2[1]+2.5
    else:
        if g_o == '0':
            xyz2[1]=xyz2[1]-1.25
        elif g_o == '1':
            xyz2[0]=xyz2[0]+1.25
        elif g_o == '-1':
            xyz2[0]=xyz2[0]-1.25
        elif g_o == '3':
            xyz2[1]=xyz2[1]+1.25
    return xyz2    
    
# new_oreint  is global orientation of the block to be loaded on that path, not the path,(opposite path)
# (last_global_orientation,path_orientation): new_orient
#path orientation is defined here 0 = south path ,    1= left path in respect to tunnel  0,0,0 point.
new_orientation = {('0','0'):'0', ('0','1'):'-1', ('0','-1'):'1',
                  ('1','0'):'1', ('1','1'):'3', ('1','-1'):'0',
                  ('-1','0'):'-1', ('-1','1'):'0', ('-1','-1'):'3',
                  ('3','0'):'3', ('3','1'):'1', ('3','-1'):'-1',}   
    
def collision_test(small_array,global_arrays,correction):
    double_path = True
    
    if isinstance(small_array[0][0], np.ndarray):
        rect= small_array
        rect1x = [rect[0][0][0]+correction[0],rect[0][0][1]+correction[0]]
        rect1y = [rect[0][1][0]+correction[1],rect[0][1][1]+correction[1]]
        
        rect2x = [rect[1][0][0]+correction[0],rect[1][0][1]+correction[0]]
        rect2y = [rect[1][1][0]+correction[1],rect[1][1][1]+correction[1]]
        
    else:
        double_path = False
        rect = small_array
        rect1x = [rect[0][0]+correction[0],rect[0][1]+correction[0]]
        rect1y = [rect[1][0]+correction[1],rect[1][1]+correction[1]]
        
    for block in global_arrays:
        blockx = [block[0][0],block[0][1]]
        blocky = [block[1][0],block[1][1]] 
        
      
        if double_path== True:
            if (blockx[1] > rect1x[0] and blockx[1] < rect1x[1]) or (blockx[0] > rect1x[0] and blockx[0]< rect1x[1]):
                x_match = True
            else:
                x_match = False
            if (blocky[1] > rect1y[0] and blocky[1] < rect1y[1]) or (blocky[0] > rect1y[0] and blocky[0] < rect1y[1]):
                y_match = True
            else:
                y_match = False
            if x_match and y_match:
                return 1
       
    
            if (blockx[1] > rect2x[0] and blockx[1] < rect2x[1]) or (blockx[0] > rect2x[0] and blockx[0]< rect2x[1]):
                x_match = True
            else:
                x_match = False
            if (blocky[1] > rect2y[0] and blocky[1] < rect2y[1]) or (blocky[0] > rect2y[0] and blocky[0] < rect2y[1]):
                y_match = True
            else:
                y_match = False
            if x_match and y_match:
                return 1
            
        else:
            if (blockx[1] > rect1x[0] and blockx[1] < rect1x[1]) or (blockx[0] > rect1x[0] and blockx[0]< rect1x[1]):
                x_match = True
            else:
                x_match = False
            if (blocky[1] > rect1y[0] and blocky[1] < rect1y[1]) or (blocky[0] > rect1y[0] and blocky[0] < rect1y[1]):
                y_match = True
            else:
                y_match = False
            if x_match and y_match:
                return 1
    return 0
            
    

In [5]:
tunnel_options = [key for key in tunnels.keys()]
xyz = np.array([0,0,0])
orientation = "0"
tunnel_location = {'LeftRight':'tunnel5.urdf', 'DownRight':'tunnel4.urdf','Downleft':'tunnel3.urdf',
                  'CrossWay':'tunnel2.urdf','Straight':'tunnel1.urdf','End':'tunnel6.urdf'}
orientation_dic = {'0':0,'1':1.5708, '-1':-1.5708, '3':3.1416}
blocks_inplace= [[[-20,20],[-20,-21]],[[-20,20],[20,21]],[[20,21],[-20,20]],[[-20,-21],[-20,20]]]
orientation_end ={"0":np.array([-1.5708,0,0]),
             "1":np.array([0,1.5708,0]),
             "-1":np.array([0,-1.5708,0]),
             "3":np.array([1.5708,0,0])} 
path_dic = {'Path_one':'0', 'Path_two':'1', 'Path_three':'-1'}
# where orientation is 0 ,1 ,-1, 3, and is global orientation.
def build(xyz,orientation):
    global blocks_inplace
    global tunnel_location
    global orientation_dic
    global new_orientation
    test_results = []
    for key in tunnel_options:
        print(xyz)
        #here add 2.5 or 1.5
        correction = xyz[:]
        correction= correct_position(orientation,key,correction)
        print(xyz)
        test= collision_test(tunnels[key]['Block_relative_coordinates'][orientation],blocks_inplace,correction)
        test_results.append([test,key])
        
    useable_tunnels = []
    for result in test_results:
        if result[0] ==0:
            useable_tunnels.append(result[1])
        
    if len(useable_tunnels)==0:
        xyz[2]+= 1.25
        p.loadURDF("tunnel6.urdf", basePosition = xyz, baseOrientation=p.getQuaternionFromEuler(orientation_end[orientation]))
    else:   
        key1 = useable_tunnels[randint(0,len(useable_tunnels))]
        print(xyz)
        #here add 2.5 or 1.5 to y or x to re-orient 
        correction1 = correct_position(orientation,key1,xyz)
        print(xyz)
        p.loadURDF(tunnel_location[key1], basePosition = correction1, baseOrientation=p.getQuaternionFromEuler((0,0,orientation_dic[orientation])))
        if key1 == 'Straight':
            x1 = tunnels[key1]['Block_relative_coordinates'][orientation][0][0] +correction1[0]
            x2 = tunnels[key1]['Block_relative_coordinates'][orientation][0][1] +correction1[0]
            y1 = tunnels[key1]['Block_relative_coordinates'][orientation][1][0] +correction1[1]
            y2 = tunnels[key1]['Block_relative_coordinates'][orientation][1][1] +correction1[1]
            blocks_inplace.append([[x1,x2],[y1,y2]])
        
        else:
            x1 = tunnels[key1]['Block_relative_coordinates'][orientation][0][0][0] +correction1[0]
            x2 = tunnels[key1]['Block_relative_coordinates'][orientation][0][0][1] +correction1[0]
            y1 = tunnels[key1]['Block_relative_coordinates'][orientation][0][1][0] +correction1[1]
            y2 = tunnels[key1]['Block_relative_coordinates'][orientation][0][1][1] +correction1[1]
            blocks_inplace.append([[x1,x2],[y1,y2]])
            
            x1 = tunnels[key1]['Block_relative_coordinates'][orientation][1][0][0] +correction1[0]
            x2 = tunnels[key1]['Block_relative_coordinates'][orientation][1][0][1] +correction1[0]
            y1 = tunnels[key1]['Block_relative_coordinates'][orientation][1][1][0] +correction1[1]
            y2 = tunnels[key1]['Block_relative_coordinates'][orientation][1][1][1] +correction1[1]
            blocks_inplace.append([[x1,x2],[y1,y2]])
            
        all_paths_available = [i for i in tunnels[key1]['Orientation_Characteristics'].keys()] 
        #random.shuffle(all_paths_available)
        for path in all_paths_available: 
            path_orientation = path_dic[path]
            n_orientation = new_orientation[(orientation,path_orientation)]
            path_xyz = correction1[:]
            path_xyz[0]+= tunnels[key1]['Orientation_Characteristics'][path][n_orientation][0]
            path_xyz[1]+= tunnels[key1]['Orientation_Characteristics'][path][n_orientation][1]
            #adjust orientation 
            build(path_xyz,n_orientation)
        
        

In [6]:
build([0,0,0],'0')

[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, 0, 0]
[0, -1.25, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -7.5, 0]
[0.0, -8.75, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-2.5, -5.0, 0]
[-5.0, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-7.5, -5.0, 0]
[-8.75, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-15.0, -5.0, 0]
[-16.25, -5.0, 0]
[-20.0, -7.5, 0]
[-20.0, -7.5, 0]
[-20.0, -7.5, 0]
[-20.0, -7.5, 0]
[-20.0, -7.5, 0]
[-20.0, -7.5, 0]
[-2

[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -205.0, 0]
[-85.0, -206.25, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -212.5, 0]
[-85.0, -213.75, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -220.0, 0]
[-85.0, -221.25, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -227.5, 0]
[-85.0, -230.0, 0]
[-85.0, -232.5, 0]
[-85.0, -232.5, 0]
[-85.0, -232.5, 0]
[-85.0, -232.5, 0]
[-85.0, -

[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -370.0, 0]
[-150.0, -371.25, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -377.5, 0]
[-150.0, -378.75, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -385.0, 0]
[-150.0, -386.25, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -392.5, 0]
[-150.0, -393.75, 0]
[-150.0, -400.0, 0]
[-150.0, -400.0, 0]
[-150.0, -400.0, 0]
[-150.0, -400.0, 0]
[-150.0, -400.0, 0]
[-150.0, -400.0,

[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-247.5, -507.5, 0]
[-248.75, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-255.0, -507.5, 0]
[-256.25, -507.5, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -510.0, 0]
[-260.0, -511.25, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -517.5, 0]
[-260.0, -518.75, 0]
[-260.0, -525.0, 0]
[-260.0, -525.0,

error: Not connected to physics server.